<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Pole-Velocity-Error" data-toc-modified-id="Pole-Velocity-Error-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Pole Velocity Error</a></span></li><li><span><a href="#Neural-Networks" data-toc-modified-id="Neural-Networks-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Neural Networks</a></span></li></ul></div>

In [1]:
#Loading the required libreries
import gym
import numpy as np
import xgboost
import itertools
import math
import pandas as pd
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.model_selection import cross_val_score, StratifiedKFold

In [2]:
#Genera un expand_grid para hacer validacion cruzada
def expand_grid(*itrs):
   product = list(itertools.product(*itrs))
   return pd.DataFrame({'Var{}'.format(i+1):[x[i] for x in product] for i in range(len(itrs))})

In [2]:
#My envorinement
my_env = gym.make('CartPole-v0')
my_box = my_env.observation_space

In [3]:
#Reset the game, the car in this case
my_env.reset()

array([-0.00061168, -0.04667878,  0.04648284, -0.04437114])

In [13]:
#Exercise: Determine how many steps, on average, are taken, when
#actions are randomly sampled
steps_array = []
for i in range(100):
    my_env.reset()
    my_env.render()
    done = False
    steps = 0
    while not done:
        obs, rew, done, _ = my_env.step(action=my_env.action_space.sample())
        steps += 1
    steps_array.append(steps)
my_env.close()

print('AVERAGE OF STEPS:', np.mean(steps_array))
print('STD OF STEPS:', np.round(np.std(steps_array), 2))

AVERAGE OF STEPS: 21.5
STD OF STEPS: 11.42


## Pole Velocity Error

In [11]:
#Exercise: Determine how many steps, on average, are taken, when
#actions are randomly sampled
steps_array = []
for i in range(5000):
    my_env.reset()
    my_env.render()
    action = my_env.action_space.sample()
    previous_error = -math.inf
    done = False
    steps = 0
    while not done:
        obs, rew, done, _ = my_env.step(action=action)
        steps += 1
        new_error = error_function(obs)
        if new_error < previous_error:
            previous_error = new_error
            pass
        else:
            previous_error = new_error
            action = abs(action-1)
            
    steps_array.append(steps)
my_env.close()
print('AVERAGE OF STEPS:', np.round(np.mean(steps_array), 2))
print('STD OF STEPS:', np.round(np.std(steps_array), 2))

AVERAGE OF STEPS: 158.51
STD OF STEPS: 22.18


In [5]:
#Define an error function using the pole velocity as the unique parameter to penalize the model
def error_function(obs):
    pos, vel, angle, pole_vel = obs
    error = (pole_vel**2)
    return error

## Neural Networks

In [212]:
data = pd.DataFrame({'POSITION': [], 'CAR_VEL': [], 'ANGLE': [], 'POL_VEL': [], 'ACTION': [], 'STATE': []})

#Exercise: Determine how many steps, on average, are taken, when
#actions are randomly sampled
steps_array = []
counter = 0

for i in range(10000):
    obs = my_env.reset()
    done = False
    steps = 0
    while not done:
        action = my_env.action_space.sample()
        new_obs, rew, done, _ = my_env.step(action=action)
        
        value = 1
        if done:
            value = 0
        
        new_data = pd.DataFrame({'POSITION': [obs[0]], 'CAR_VEL': [obs[1]], 'ANGLE': [obs[2]],
                                 'POL_VEL': [obs[3]], 'ACTION': [action], 'STATE': [value]})
        
        obs = new_obs
        data = pd.concat([data, new_data])
        steps += 1
    steps_array.append(steps)
    if (i+1) % 1000 == 0:
        print('ITERATION:', i+1, ' - DATA SHAPE:', data.shape)        

print('AVERAGE OF STEPS:', np.mean(steps_array))

ITERATION: 1000  - DATA SHAPE: (22442, 6)
ITERATION: 2000  - DATA SHAPE: (45017, 6)
ITERATION: 3000  - DATA SHAPE: (67049, 6)
ITERATION: 4000  - DATA SHAPE: (89209, 6)
ITERATION: 5000  - DATA SHAPE: (112028, 6)
ITERATION: 6000  - DATA SHAPE: (135285, 6)
ITERATION: 7000  - DATA SHAPE: (157105, 6)
ITERATION: 8000  - DATA SHAPE: (179401, 6)
ITERATION: 9000  - DATA SHAPE: (202025, 6)
ITERATION: 10000  - DATA SHAPE: (224204, 6)
AVERAGE OF STEPS: 22.4204


In [222]:
#Reset the index
data = data.reset_index(drop=True)

In [242]:
predictors = data[['POSITION', 'CAR_VEL', 'ANGLE', 'POL_VEL', 'ACTION']]
outcome = data['STATE']

#Separamos los datos en entrenamiento y test
x_train, x_test, y_train, y_test = train_test_split(predictors, outcome, test_size=0.3, random_state=1000,
                                                    stratify=outcome) 


clf = MLPClassifier(solver='lbfgs', alpha=1e-5, hidden_layer_sizes=(10, 5), random_state=1)

In [243]:
#Train the MLP 
clf.fit(x_train, y_train)

MLPClassifier(activation='relu', alpha=1e-05, batch_size='auto', beta_1=0.9,
              beta_2=0.999, early_stopping=False, epsilon=1e-08,
              hidden_layer_sizes=(10, 5), learning_rate='constant',
              learning_rate_init=0.001, max_fun=15000, max_iter=200,
              momentum=0.9, n_iter_no_change=10, nesterovs_momentum=True,
              power_t=0.5, random_state=1, shuffle=True, solver='lbfgs',
              tol=0.0001, validation_fraction=0.1, verbose=False,
              warm_start=False)

In [244]:
#Predice para el test y entrega las principales metricas
expected = y_test
predicted = clf.predict(x_test)
print("Classification report for classifier %s:\n%s\n"
      % (clf, metrics.classification_report(expected, predicted)))

Classification report for classifier MLPClassifier(activation='relu', alpha=1e-05, batch_size='auto', beta_1=0.9,
              beta_2=0.999, early_stopping=False, epsilon=1e-08,
              hidden_layer_sizes=(10, 5), learning_rate='constant',
              learning_rate_init=0.001, max_fun=15000, max_iter=200,
              momentum=0.9, n_iter_no_change=10, nesterovs_momentum=True,
              power_t=0.5, random_state=1, shuffle=True, solver='lbfgs',
              tol=0.0001, validation_fraction=0.1, verbose=False,
              warm_start=False):
              precision    recall  f1-score   support

         0.0       1.00      1.00      1.00      3000
         1.0       1.00      1.00      1.00     64262

    accuracy                           1.00     67262
   macro avg       1.00      1.00      1.00     67262
weighted avg       1.00      1.00      1.00     67262




In [249]:
predictors = new_data[['POSITION', 'CAR_VEL', 'ANGLE', 'POL_VEL', 'ACTION']]
clf.predict(predictors)[0]

0.0

In [256]:
#Exercise: Determine how many steps, on average, are taken, when
#actions are randomly sampled
steps_array = []
for i in range(100):
    obs = my_env.reset()
    done = False
    steps = 0
    while not done:
        
        new_data_action_zero = pd.DataFrame({'POSITION': [obs[0]], 'CAR_VEL': [obs[1]], 'ANGLE': [obs[2]],
                                             'POL_VEL': [obs[3]], 'ACTION': [0]})
        new_data_action_one = pd.DataFrame({'POSITION': [obs[0]], 'CAR_VEL': [obs[1]], 'ANGLE': [obs[2]],
                                             'POL_VEL': [obs[3]], 'ACTION': [1]})
        
        pred_zero = clf.predict(new_data_action_zero)[0]
        pred_one = clf.predict(new_data_action_one)[0]
        print(pred_zero, pred_one)
        
        if pred_zero > pred_one:
            obs, rew, done, _ = my_env.step(action=0)
        else:
            obs, rew, done, _ = my_env.step(action=1)
        
        steps += 1
            
    steps_array.append(steps)

print('AVERAGE OF STEPS:', np.round(np.mean(steps_array), 2))
print('STD OF STEPS:', np.round(np.std(steps_array), 2))

1.0 1.0
1.0 1.0
1.0 1.0
1.0 1.0
1.0 1.0
1.0 1.0
1.0 1.0
0.0 0.0
1.0 1.0
1.0 1.0
1.0 1.0
1.0 1.0
1.0 1.0
1.0 1.0
1.0 1.0
1.0 1.0
1.0 1.0
0.0 0.0
1.0 1.0
1.0 1.0
1.0 1.0
1.0 1.0
1.0 1.0
1.0 1.0
1.0 1.0
1.0 1.0
1.0 1.0
0.0 0.0
1.0 1.0
1.0 1.0
1.0 1.0
1.0 1.0
1.0 1.0
1.0 1.0
1.0 1.0
1.0 1.0
1.0 1.0
0.0 0.0
1.0 1.0
1.0 1.0
1.0 1.0
1.0 1.0
1.0 1.0
1.0 1.0
1.0 1.0
0.0 0.0
1.0 1.0
1.0 1.0
1.0 1.0
1.0 1.0
1.0 1.0
1.0 1.0
1.0 1.0
0.0 0.0
1.0 1.0
1.0 1.0
1.0 1.0
1.0 1.0
1.0 1.0
1.0 1.0
1.0 1.0
1.0 1.0
1.0 1.0
0.0 0.0
1.0 1.0
1.0 1.0
1.0 1.0
1.0 1.0
1.0 1.0
1.0 1.0
1.0 1.0
1.0 1.0
0.0 0.0
1.0 1.0
1.0 1.0
1.0 1.0
1.0 1.0
1.0 1.0
1.0 1.0
1.0 1.0
1.0 1.0
0.0 0.0
1.0 1.0
1.0 1.0
1.0 1.0
1.0 1.0
1.0 1.0
1.0 1.0
1.0 1.0
1.0 1.0
0.0 0.0
1.0 1.0
1.0 1.0
1.0 1.0
1.0 1.0
1.0 1.0
1.0 1.0
1.0 1.0
1.0 1.0
0.0 0.0
1.0 1.0
1.0 1.0
1.0 1.0
1.0 1.0
1.0 1.0
1.0 1.0
1.0 1.0
1.0 1.0
0.0 0.0
1.0 1.0
1.0 1.0
1.0 1.0
1.0 1.0
1.0 1.0
1.0 1.0
1.0 1.0
1.0 1.0
1.0 1.0
0.0 0.0
1.0 1.0
1.0 1.0
1.0 1.0
1.0 1.0
1.0 1.0
1.0 1.0


In [245]:
#Entrena el arbol boosteado
#Computa la precisión sobre la muestra de entrenamiento y la de validacion
learning_rates = [0.25, 0.5, 1]
max_depths = [3, 5, 7, 10]
subsamples = [1]
colsample_bytree = [1]

iterations = expand_grid(learning_rates, max_depths, subsamples, colsample_bytree)
iterations.columns = ['Learning Rate', 'Max Depth', 'Row Subsample', 'Col Subsample']
iterations = iterations.where((pd.notnull(iterations)), None)
iterations['Score CV'] = 0.0

for it in range(len(iterations)):
    kf = StratifiedKFold(n_splits=3, random_state=1608, shuffle=True)
    xg_model = xgboost.XGBClassifier(learning_rate=iterations['Learning Rate'][it],
                                     max_depth=iterations['Max Depth'][it]
                                     #subsample=iterations['Row Subsample'][it], 
                                     #colsample_bytree=iterations['Col Subsample'][it]
                                    )
    score = cross_val_score(xg_model, x_train, y_train, cv=kf)
    
    iterations.at[it, 'Score CV'] = score.mean()
    print('Hecha Combinacion', it+1, 'de', len(iterations), '-', 'Score CV:', score.mean())

Hecha Combinacion 1 de 12 - Score CV: 0.9988403359202763


KeyboardInterrupt: 